# Convolutional Neural Network

### Importing the libraries

In [1]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
tf.__version__

'2.19.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,  # Ölçeklendirme 0-255 olan piksel değerleri 0-1 arasında ölçeklendirilir.
    rotation_range = 20, # Göreseli merkezzde 0-20 arasında rastgele bir açıyla döndürür.
    #shear_range=0.2,     # Paralel kayma uygular. Üs kısım sabit tutulup, alt kısım sağa kaydırılır.
    zoom_range = 0.2,    # Zoom(yakınlaştırma) yapılır.
    horizontal_flip = True # Yatayda ayna gibi çevirmeyi ektif eder.
)

training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size = (64,64), # Görüntülerin boyutu verilen değerler olarak ayarlanır.
    batch_size = 32,        # Modelin her seferinde 32 resim üzerinde ileri ve geri yayılım yapmasını sağlar.
    class_mode = 'binary'   # iki sınıflı sınıflandırmayı belirtir.
)

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(
    rescale = 1. /255
)

test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size = (64,64),
    batch_size = 32,
    class_mode = 'binary'
)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = Sequential()

### Step 1 - Convolution

In [6]:
cnn.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = (64, 64, 3)))

C:\Users\BaranBey\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [7]:
cnn.add(MaxPooling2D(pool_size = (2,2), strides = 2))

### Adding a second convolutional layer

In [8]:
cnn.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
cnn.add(MaxPooling2D(pool_size = (2,2), strides = 2))

### Adding a third convolutional layer

In [9]:
cnn.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu'))
cnn.add(MaxPooling2D(pool_size = (2,2), strides = 2))

### Step 3 - Flattening

In [10]:
cnn.add(Flatten())

### Step 4 - Full Connection

In [11]:
cnn.add(Dense(units = 128, activation = 'relu'))
cnn.add(Dropout(0.5))  # Overfitting’e karşı

### Step 5 - Output Layer

In [12]:
cnn.add(Dense(units = 1, activation = 'sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [13]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25


C:\Users\BaranBey\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 26s 97ms/step - accuracy: 0.5252 - loss: 0.6908 - val_accuracy: 0.6540 - val_loss: 0.6365
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 26s 102ms/step - accuracy: 0.6638 - loss: 0.6203 - val_accuracy: 0.7020 - val_loss: 0.5847
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 26s 104ms/step - accuracy: 0.6962 - loss: 0.5797 - val_accuracy: 0.7280 - val_loss: 0.5405
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 26s 103ms/step - accuracy: 0.7129 - loss: 0.5580 - val_accuracy: 0.7555 - val_loss: 0.5003
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 98ms/step - accuracy: 0.7391 - loss: 0.5282 - val_accuracy: 0.7710 - val_loss: 0.4763
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 99ms/step - accuracy: 0.7541 - loss: 0.5064 - val_accuracy: 0.7915 - val_loss: 0.4541
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 100ms/step - accuracy: 0.7665 - loss: 0.4847 - val_accuracy: 0.7535 - val_loss: 0.4928
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 101ms/step - accuracy: 0.7722 - loss: 0.4779 - val_ac

## Part 4 - Making a single prediction

In [95]:
test_image = image.load_img('dataset/single_prediction/dog-1869167_640.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = cnn.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


In [96]:
print(prediction)

dog


In [97]:
print(training_set.class_indices)

{'cats': 0, 'dogs': 1}
